In [60]:
!pip install metar

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [76]:
import time
import datetime
from datetime import date
import pandas as pd
import requests
from metar import Metar
from datetime import date, timedelta, datetime
import time
from tqdm import tqdm
import numpy as np

## V1

In [84]:
all_airports = requests.get("https://www.flightradar24.com/_json/airports.php", headers= {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36 OPR/51.0.2830.55"
    }).json()["rows"]
all_airports = pd.DataFrame(all_airports)


In [77]:
dep = pd.read_csv("svo_dep_aeroflot_rossiya_05_06_2022.csv")
arr = pd.read_csv("svo_arr_aeroflot_rossiya_05_06_2022.csv")

In [107]:
airports = np.concatenate([dep.arrival_latest_airportCode.unique(), 
                           arr.departure_airportCode.unique(),
                           np.array(["SVO"])
                          ])
airports = np.unique(airports)
airports = pd.DataFrame(airports, columns=["iata"]).merge(all_airports[["iata", "icao"]])
airports

,iata,icao
0,ABA,UNAA
1,AER,URSS
2,AKX,UATT
3,ALA,UAAA
4,ARH,ULAA
...,...,...
73,ULV,UWLL
74,UUS,UHSS
75,VOG,URWW
76,VVO,UHWW


In [108]:
df = pd.DataFrame()

for airport in tqdm(airports["icao"].tolist()):
    url=f"https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station={airport}&data=all&year1=2022&month1=4&day1=29&year2=2022&month2=7&day2=2&tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=empty&trace=0.0001&direct=no&report_type=3"
    qq=pd.read_csv(url)
    df = pd.concat([df, qq]).reset_index().drop(["index"], axis=1)
    

100%|███████████████████████████████████████████| 78/78 [02:58<00:00,  2.28s/it]


In [203]:
def get_temp(x):
    try:
        return Metar.Metar(x).temp.value("C")
    except:
        return np.NaN

def get_wind_speed(x):
    try:
        return Metar.Metar(x).wind_speed.value("KMH")
    except:
        return np.NaN
    
def get_visibility(x):
    try:
        return Metar.Metar(x).vis.value("M")
    except:
        return np.NaN   
    
def get_sky_coverage(x):
    try:
        obs = Metar.Metar(x).sky_conditions()
        if not obs:
            return -1
        elif obs.find('scattered') != -1:
            return 0.4
        elif obs.find('broken') != -1:
            return 1
        elif obs.find('overcast') != -1:
            return 0.6
        elif obs.find('few') != -1:
            return 0.2
        elif obs.find('indefin') != -1:
            return 
        elif obs.find('clear') != -1:
            return 0
        else:
            return -1
    except:
        return np.NaN

In [208]:
weather = df.copy(True)

In [209]:
weather["temp"] = weather["metar"].apply(lambda x: get_temp(x))
weather["wind_speed"] = weather["metar"].apply(lambda x: get_wind_speed(x))
weather["visibility"] = weather["metar"].apply(lambda x: get_visibility(x))
weather["sky_coverage"] = weather["metar"].apply(lambda x: get_sky_coverage(x))

In [210]:
weather = weather.rename(columns={"station": "icao", "valid": "datetime"})

In [223]:
import datetime

def get_datetime(x):
    try:
        return datetime.datetime.strptime(x, "%Y-%m-%d %H:%M")
    except:
        return datetime.datetime.strptime(x, "%Y-%m-%d%H:%M")

In [225]:
weather["datetime"] = weather["datetime"].apply(lambda x: get_datetime(x))

In [229]:
weather.to_csv("svo_weather.csv", index=False)

## Test examples 

In [42]:


def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2022, 4, 29)
end_date = date(2022, 5, 2)
counter = 0
weather = pd.DataFrame()
for airport in tqdm(top_airports.icao.tolist()):
    time.sleep(10)
    for single_date in daterange(start_date, end_date):
        single_datetime = int(single_date.strftime("%s"))
        r = requests.get(f"https://navlost.eu/api/reports/metar/{airport}?from={single_datetime}&limit=100&until={single_datetime+3600*24}",
                    headers= {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36 OPR/51.0.2830.55"
        }).json()
            
        print(r)
        qq = pd.DataFrame(r)
        weather = pd.concat([weather, qq]).reset_index().drop(["index"], axis=1)
    
    

  0%|                                                    | 0/77 [00:10<?, ?it/s]

{'msg': 'Too many requests'}


ValueError: If using all scalar values, you must pass an index

In [14]:
weather

,type,icao,tstamp,report,temp,dewpt,wind,visibility,pressure,sky_conditions
0,metar,UUDD,2022-04-29T21:00:00.000Z,UUDD 292100Z 27003MPS 9999 FEW033 03/00 Q1022 ...,3.0 C,0.0 C,W at 6 knots,greater than 10000 meters,1022.0 mb,a few clouds at 3300 feet
1,metar,UUDD,2022-04-29T20:00:00.000Z,UUDD 292000Z 30006MPS 9999 SCT020CB 05/00 Q102...,5.0 C,0.0 C,WNW at 12 knots,greater than 10000 meters,1022.0 mb,scattered cumulonimbus at 2000 feet
2,metar,UUDD,2022-04-29T19:00:00.000Z,UUDD 291900Z 29007MPS 260V320 9999 -SHRA BKN02...,6.0 C,-1.0 C,W to NW at 14 knots,greater than 10000 meters,1022.0 mb,broken cumulonimbus at 2600 feet
3,metar,UUDD,2022-04-29T18:00:00.000Z,UUDD 291800Z 25002MPS 200V280 CAVOK 08/M06 Q10...,8.0 C,-6.0 C,SSW to W at 4 knots,10000 meters,1021.0 mb,
4,metar,UUDD,2022-04-29T17:00:00.000Z,UUDD 291700Z 23003MPS CAVOK 09/M07 Q1021 R32L/...,9.0 C,-7.0 C,SW at 6 knots,10000 meters,1021.0 mb,
...,...,...,...,...,...,...,...,...,...,...
69,metar,UUDD,2022-05-01T01:00:00.000Z,UUDD 010100Z 18002MPS CAVOK M02/M04 Q1024 R32L...,-2.0 C,-4.0 C,S at 4 knots,10000 meters,1024.0 mb,
70,metar,UUDD,2022-05-01T00:00:00.000Z,UUDD 010000Z 19002MPS CAVOK M01/M04 Q1024 R32L...,-1.0 C,-4.0 C,S at 4 knots,10000 meters,1024.0 mb,
71,metar,UUDD,2022-04-30T23:00:00.000Z,UUDD 302300Z 16001MPS CAVOK M01/M04 Q1024 R32L...,-1.0 C,-4.0 C,SSE at 2 knots,10000 meters,1024.0 mb,
72,metar,UUDD,2022-04-30T22:00:00.000Z,UUDD 302200Z VRB01MPS CAVOK M00/M04 Q1024 R32L...,-0.0 C,-4.0 C,variable at 2 knots,10000 meters,1024.0 mb,


In [15]:
top_airports = ['HMA', 'AER', 'MRV', 'GOJ', 'LED', 'KGD', 'ALA', 'SCW', 'MMK',
       'REN', 'SGC', 'TJM', 'OMS', 'MQF', 'PEE', 'NBC', 'NJC', 'KJA',
       'UFA', 'OSS', 'KUF', 'SVX', 'IAR', 'NQZ', 'ARH', 'ULV', 'CEK',
       'FRU', 'VOG', 'KZN', 'NOZ', 'KEJ', 'TOF', 'IJK', 'ASF', 'EVN',
       'VVO', 'OVB', 'NUX', 'KHV', 'ABA', 'IKT', 'GSV', 'RGK', 'PKC',
       'UUS', 'MSQ', 'PEZ', 'GDX', 'GRV', 'MCX', 'OSW', 'STW', 'SCO',
       'GYD', 'KGF', 'YKS', 'GUW', 'CMB', 'IGT', 'IKA', 'KSN', 'AKX',
       'SKD', 'DEL', 'AYT', 'IST', 'BHK', 'UGC', 'MLE', 'SVO', 'TAS',
       'DYR', 'OGZ', 'BAX', 'IKU', 'NAL']

In [26]:
top_airports = pd.DataFrame(top_airports, columns=["iata"]).merge(airports[["iata", "icao"]])

In [18]:
airports = requests.get("https://www.flightradar24.com/_json/airports.php", headers= {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36 OPR/51.0.2830.55"
    }).json()["rows"]
airports = pd.DataFrame(airports)


In [47]:
r = requests.get("https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=UUDD&data=all&year1=2022&month1=4&day1=29&year2=2022&month2=7&day2=2&tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=empty&trace=0.0001&direct=no&report_type=3", headers= {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36 OPR/51.0.2830.55"
    })



In [53]:
pd.DataFrame(r.text)

ValueError: DataFrame constructor not properly called!

In [56]:
for airport in tqdm(top_airports.icao.tolist()):
    url=f"https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station={airport}&data=all&year1=2022&month1=4&day1=29&year2=2022&month2=7&day2=2&tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=empty&trace=0.0001&direct=no&report_type=3"
    c=pd.read_csv(url)

 31%|█████████████▍                             | 24/77 [00:32<01:11,  1.34s/it]


KeyboardInterrupt: 